## 서울특별시 공공자전거 실시간 대여정보

API 제공 URL : http://data.seoul.go.kr/dataList/datasetView.do?infId=OA-15493&srvType=A&serviceKind=1

#### response example

In [5]:
ex = {"rentBikeStatus":{"list_total_count":2,"RESULT":{"CODE":"INFO-000","MESSAGE":"정상 처리되었습니다."},"row":[{"rackTotCnt":"10","stationName":"108. 서교동 사거리","parkingBikeTotCnt":"0","shared":"0","stationLatitude":"37.552746","stationLongitude":"126.918617","stationId":"ST-10"},{"rackTotCnt":"15","stationName":"503. 더샵스타시티 C동 앞","parkingBikeTotCnt":"3","shared":"20","stationLatitude":"37.536667","stationLongitude":"127.073593","stationId":"ST-100"}]}}
ex

{'rentBikeStatus': {'list_total_count': 2,
  'RESULT': {'CODE': 'INFO-000', 'MESSAGE': '정상 처리되었습니다.'},
  'row': [{'rackTotCnt': '10',
    'stationName': '108. 서교동 사거리',
    'parkingBikeTotCnt': '0',
    'shared': '0',
    'stationLatitude': '37.552746',
    'stationLongitude': '126.918617',
    'stationId': 'ST-10'},
   {'rackTotCnt': '15',
    'stationName': '503. 더샵스타시티 C동 앞',
    'parkingBikeTotCnt': '3',
    'shared': '20',
    'stationLatitude': '37.536667',
    'stationLongitude': '127.073593',
    'stationId': 'ST-100'}]}}

#### response 정보

1	rackTotCnt	거치대개수

2	stationName	대여소이름

3	parkingBikeTotCnt	자전거주차총건수

4	shared	거치율

5	stationLatitude	위도

6	stationLongitude	경도

7	stationId	대여소ID

### API를 활용하여 실시간 운영중인 대여소 추출

In [1]:
import requests
import json
import pandas as pd

In [2]:
# 한 번에 최대 1000개 정보 제공
auth_key = '6f506b796d68747332396957746263'
response = requests.get(f'http://openapi.seoul.go.kr:8088/{auth_key}/json/bikeList/1/1000')

In [3]:
bike_rent_info = json.loads(response.text)['rentBikeStatus']['row']
bike_rent_df1 = pd.DataFrame(bike_rent_info)

In [7]:
# 1001 ~
response = requests.get(f'http://openapi.seoul.go.kr:8088/{auth_key}/json/bikeList/1001/2000')
bike_rent_info = json.loads(response.text)['rentBikeStatus']['row']
bike_rent_df2 = pd.DataFrame(bike_rent_info)

In [8]:
bike_rent_df = pd.concat([bike_rent_df1, bike_rent_df2], ignore_index=True) # 데이터 결합

In [9]:
bike_rent_df.head()

,parkingBikeTotCnt,rackTotCnt,shared,stationId,stationLatitude,stationLongitude,stationName
0,9,5,180,ST-3,37.549561,126.905754,101. (구)합정동 주민센터
1,4,20,20,ST-4,37.555649,126.910629,102. 망원역 1번출구 앞
2,3,15,20,ST-1040,37.549999,127.174690,1023. 한국종합기술사옥 앞
3,6,10,60,ST-1041,37.529251,127.123108,1024. 강동구청 앞
4,16,10,160,ST-1042,37.546841,127.172516,1025. 상일초등학교


In [12]:
# station Name에서 번호만 추출
def extract_content_id(station_name): 
    if '.' in station_name:
        res = station_name.split('.')[0]
    else:
        res = 0
    return res

bike_rent_df['CONTENT_ID'] = bike_rent_df['stationName'].map(extract_content_id)

In [14]:
# station Name에서 번호 제거
def extract_name(station_name):
    return station_name.split('.')[-1].lstrip()

bike_rent_df['stationName'] = bike_rent_df['stationName'].map(extract_name)

In [15]:
bike_rent_df.head()

,parkingBikeTotCnt,rackTotCnt,shared,stationId,stationLatitude,stationLongitude,stationName,CONTENT_ID
0,9,5,180,ST-3,37.549561,126.905754,(구)합정동 주민센터,101
1,4,20,20,ST-4,37.555649,126.910629,망원역 1번출구 앞,102
2,3,15,20,ST-1040,37.549999,127.174690,한국종합기술사옥 앞,1023
3,6,10,60,ST-1041,37.529251,127.123108,강동구청 앞,1024
4,16,10,160,ST-1042,37.546841,127.172516,상일초등학교,1025


In [16]:
bike_rent_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1529 entries, 0 to 1528
Data columns (total 8 columns):
parkingBikeTotCnt    1529 non-null object
rackTotCnt           1529 non-null object
shared               1529 non-null object
stationId            1529 non-null object
stationLatitude      1529 non-null object
stationLongitude     1529 non-null object
stationName          1529 non-null object
CONTENT_ID           1529 non-null object
dtypes: object(8)
memory usage: 95.6+ KB


2019-05-22 기준 1529 개